In [1]:
import os
from glob import glob

In [2]:
train_fns = glob(os.path.join('data', 'sentiment', 'train', '*', '*.txt'))
test_fns = glob(os.path.join('data', 'sentiment', 'test', '*', '*.txt'))

In [3]:
from tqdm import tqdm
max_length = 0


for fn in tqdm(train_fns):
    with open(fn) as f:
        lines = f.readlines()
        # lines is lies of len 1
        if len(lines) != 1: 
            print(lines)
            print('not of len 1')
            print(1/0)
        len_text = len(lines[0].split(' '))
        if len_text > max_length: max_length = len_text


print(max_length)
max_length = max_length * 2
print(f'max token length will be set to {max_length}')

  6%|▋         | 1581/25000 [00:04<01:13, 316.74it/s]


KeyboardInterrupt: 

In [14]:
from nlp import datasets
import importlib
importlib.reload(datasets)
from transformers import BertTokenizer
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# for bert
max_length = 512

# create datasets
train_dataset = datasets.SentimentDataset(train_fns, tokenizer, max_length=max_length)
test_dataset = datasets.SentimentDataset(test_fns[0:250], tokenizer, max_length=max_length)

# Create DataLoader
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print(f'Trainset has {len(train_dataset)} examples, testset has {len(test_dataset)}')


Trainset has 25000 examples, testset has 250


In [8]:
import torch
from torch.optim import AdamW

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# Move model to GPU if available
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)  # Learning rate of 2e-5 is typical for BERT fine-tuning
criterion = torch.nn.CrossEntropyLoss()

# Training loop
def train_model(model, train_loader, optimizer, criterion, device, num_epochs=3):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            # Get inputs from batch and move to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass: get predictions and compute loss
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss  # Loss is already calculated internally for BertForSequenceClassification
            total_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Print the average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

# Evaluate model on test set
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    total_correct = 0
    total_examples = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            # Compute accuracy
            total_correct += (predictions == labels).sum().item()
            total_examples += labels.size(0)

    accuracy = total_correct / total_examples
    print(f'Test Accuracy: {accuracy:.4f}')

# Fine-tune the model
train_model(model, train_loader, optimizer, criterion, device, num_epochs=3)

# Evaluate the fine-tuned model
evaluate_model(model, test_loader, device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.0031


KeyboardInterrupt: 

In [9]:
import joblib
joblib.dump(model, filename=os.path.join('first_test_model.pkl'))

['first_test_model.pkl']

In [16]:
import torch
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertForSequenceClassification


# Load the pre-trained BERT model for sequence classification
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set the model to evaluation mode
model.eval()

# Make sure to run on the GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model.to(device)

# Function to evaluate the model on a dataset
def evaluate_model(data_loader):
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass: get logits
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            # Get predictions (use argmax to get the index of the max logit)
            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            all_predictions.extend(predictions)
            all_labels.extend(labels.cpu().numpy())

    return all_predictions, all_labels

# Evaluate on training dataset
# print('train performance')
# train_predictions, train_labels = evaluate_model(train_loader)
# train_accuracy = accuracy_score(train_labels, train_predictions)

# Evaluate on test dataset
print('test performance')
test_predictions, test_labels = evaluate_model(test_loader)
test_accuracy = accuracy_score(test_labels, test_predictions)

# Print the results
# print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Optional: Detailed classification report
print("Classification Report (Test Set):")
print(classification_report(test_labels, test_predictions, target_names=["Negative", "Positive"]))

cuda
test performance


100%|██████████| 125/125 [00:02<00:00, 45.43it/s]


Test Accuracy: 1.0000
Classification Report (Test Set):


ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import Trainer, TrainingArguments

c:\Users\jnicolow\AppData\Local\anaconda3\envs\nlp-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


c:\Users\jnicolow\AppData\Local\anaconda3\envs\nlp-env\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jnicolow\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
texts = ["I love this!", "This is terrible."]



tokenizer.encode_plus(
    texts[0],
    add_special_tokens=True,
    max_length=3000,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True,
)
\


{'input_ids': tensor([[ 101, 1045, 2293,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}